# Feed-Forward Network (FFN)

## Introduction

Bienvenue dans ce notebook sur le **Feed-Forward Network**, le composant qui ajoute de la capacité de transformation non-linéaire dans chaque TransformerBlock!

### Objectifs pédagogiques

Dans ce notebook, vous allez:
1. Comprendre l'architecture du Feed-Forward Network
2. Implémenter FFN from scratch avec NumPy
3. Implémenter FFN avec PyTorch
4. Comparer les fonctions d'activation GELU et ReLU
5. Visualiser les différences entre GELU et ReLU
6. Valider la préservation des dimensions

### Qu'est-ce qu'un Feed-Forward Network?

Le FFN est un réseau de neurones simple appliqué **indépendamment** à chaque position de la séquence (d'où "position-wise").

**Analogie:** Imaginez que l'attention permet aux tokens de "communiquer" entre eux. Le FFN permet ensuite à chaque token de "réfléchir" individuellement sur ce qu'il a appris.

**Architecture:**
- **Couche 1:** Expansion (d_model → d_ff, typiquement d_ff = 4 × d_model)
- **Activation:** Non-linéarité (GELU ou ReLU)
- **Couche 2:** Projection (d_ff → d_model)

**Rôle dans le Transformer:**
- Après l'attention, le FFN ajoute de la capacité de transformation
- Permet au modèle d'apprendre des représentations plus complexes
- Appliqué de manière identique à chaque position ("position-wise")

## 1. Formules Mathématiques du Feed-Forward Network

### Formule Complète

$\text{FFN}(x) = \text{activation}(xW_1 + b_1)W_2 + b_2$

### Décomposition Étape par Étape

#### Étape 1: Première Transformation Linéaire (Expansion)

$h = xW_1 + b_1$

Où:
- $x \in \mathbb{R}^{d_{\text{model}}}$ : Input (un token)
- $W_1 \in \mathbb{R}^{d_{\text{model}} \times d_{\text{ff}}}$ : Première matrice de poids
- $b_1 \in \mathbb{R}^{d_{\text{ff}}}$ : Premier biais
- $h \in \mathbb{R}^{d_{\text{ff}}}$ : Représentation cachée (expanded)

**Dimensions typiques:**
- $d_{\text{model}} = 256, 512, 768$
- $d_{\text{ff}} = 4 \times d_{\text{model}} = 1024, 2048, 3072$

#### Étape 2: Activation Non-Linéaire

$h' = \text{activation}(h)$

**GELU (Gaussian Error Linear Unit):**

$\text{GELU}(x) = x \cdot \Phi(x)$

Où $\Phi(x)$ est la fonction de répartition de la loi normale standard.

**Approximation de GELU:**

$\text{GELU}(x) \approx 0.5 \cdot x \cdot \left(1 + \tanh\left(\sqrt{\frac{2}{\pi}} \left(x + 0.044715 \cdot x^3\right)\right)\right)$

**ReLU (Rectified Linear Unit):**

$\text{ReLU}(x) = \max(0, x)$

#### Étape 3: Deuxième Transformation Linéaire (Projection)

$y = h'W_2 + b_2$

Où:
- $W_2 \in \mathbb{R}^{d_{\text{ff}} \times d_{\text{model}}}$ : Deuxième matrice de poids
- $b_2 \in \mathbb{R}^{d_{\text{model}}}$ : Deuxième biais
- $y \in \mathbb{R}^{d_{\text{model}}}$ : Output (même dimension que l'input)

### Propriété Importante: Préservation de Dimension

**Input:** $(\text{batch}, \text{seq\_len}, d_{\text{model}})$

**Output:** $(\text{batch}, \text{seq\_len}, d_{\text{model}})$

Le FFN préserve les dimensions d'entrée, ce qui permet de l'utiliser dans les connexions résiduelles.

### Pourquoi GELU plutôt que ReLU?

**Avantages de GELU:**
1. **Lisse partout:** Pas de discontinuité en 0
2. **Gradient non-nul pour valeurs négatives:** Évite la "mort" des neurones
3. **Performance empirique:** Meilleurs résultats dans BERT et GPT
4. **Propriétés probabilistes:** Basé sur la distribution gaussienne

**ReLU:**
- Plus simple et rapide
- Peut "tuer" des neurones (gradient = 0 pour x < 0)
- Discontinuité en 0

In [ ]:
# Imports
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from typing import Tuple, Optional

# Ajouter le chemin vers src/
sys.path.append('../..')

# Importer nos modules
from src.architecture.feed_forward import (
    feed_forward_from_scratch,
    gelu_from_scratch,
    relu_from_scratch,
    FeedForward,
    FeedForwardSequential,
    compare_activations,
    validate_dimension_preservation
)

# Configuration pour les visualisations
plt.style.use('default')
sns.set_palette("husl")

# Seed pour la reproductibilité
np.random.seed(42)
torch.manual_seed(42)

print("✓ Imports réussis!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 2. Implémentation From Scratch (NumPy)

Commençons par implémenter le FFN avec NumPy pour comprendre chaque opération matricielle.

### 2.1 Exemple Simple avec 1 Token

In [ ]:
# Paramètres pour l'exemple
d_model = 8      # dimension du modèle (petite pour la visualisation)
d_ff = 32        # dimension cachée (4 × d_model)

# Créer un input simple (1 token)
np.random.seed(42)
x = np.random.randn(d_model)

# Créer les matrices de poids
W1 = np.random.randn(d_model, d_ff) * 0.1
b1 = np.zeros(d_ff)
W2 = np.random.randn(d_ff, d_model) * 0.1
b2 = np.zeros(d_model)

print("=" * 70)
print("EXEMPLE: Feed-Forward Network from Scratch (1 token)")
print("=" * 70)
print(f"\nConfiguration:")
print(f"  - d_model: {d_model}")
print(f"  - d_ff: {d_ff}")
print(f"  - Ratio d_ff/d_model: {d_ff/d_model}")
print(f"\nDimensions:")
print(f"  - Input x: {x.shape}")
print(f"  - W1: {W1.shape}")
print(f"  - b1: {b1.shape}")
print(f"  - W2: {W2.shape}")
print(f"  - b2: {b2.shape}")

In [ ]:
# Étape 1: Première transformation linéaire (expansion)
h = np.matmul(x, W1) + b1
print(f"\nÉtape 1: Première transformation linéaire")
print(f"  h = xW1 + b1")
print(f"  Shape: {x.shape} @ {W1.shape} + {b1.shape} -> {h.shape}")
print(f"  Dimension: {d_model} -> {d_ff} (expansion)")
print(f"  h[:5] = {h[:5]}")

# Étape 2: Activation GELU
h_activated = gelu_from_scratch(h)
print(f"\nÉtape 2: Activation GELU")
print(f"  h' = GELU(h)")
print(f"  Shape: {h.shape} -> {h_activated.shape}")
print(f"  h'[:5] = {h_activated[:5]}")
print(f"  Effet: Valeurs négatives atténuées, positives préservées")

# Étape 3: Deuxième transformation linéaire (projection)
y = np.matmul(h_activated, W2) + b2
print(f"\nÉtape 3: Deuxième transformation linéaire")
print(f"  y = h'W2 + b2")
print(f"  Shape: {h_activated.shape} @ {W2.shape} + {b2.shape} -> {y.shape}")
print(f"  Dimension: {d_ff} -> {d_model} (projection)")
print(f"  y[:5] = {y[:5]}")

print(f"\n{'='*70}")
print(f"RÉSULTAT FINAL")
print(f"{'='*70}")
print(f"  Input shape:  {x.shape}")
print(f"  Output shape: {y.shape}")
print(f"  ✓ Dimension préservée: {x.shape == y.shape}")

### 2.2 Exemple avec Batch et Séquence

In [ ]:
# Paramètres pour l'exemple avec batch
batch_size = 2
seq_len = 5
d_model = 8
d_ff = 32

# Créer l'input (batch de séquences)
np.random.seed(42)
x_batch = np.random.randn(batch_size, seq_len, d_model)

# Créer les matrices de poids
W1 = np.random.randn(d_model, d_ff) * 0.1
b1 = np.zeros(d_ff)
W2 = np.random.randn(d_ff, d_model) * 0.1
b2 = np.zeros(d_model)

print("=" * 70)
print("EXEMPLE: Feed-Forward Network from Scratch (batch + séquence)")
print("=" * 70)
print(f"\nConfiguration:")
print(f"  - batch_size: {batch_size}")
print(f"  - seq_len: {seq_len}")
print(f"  - d_model: {d_model}")
print(f"  - d_ff: {d_ff}")
print(f"\nInput shape: {x_batch.shape}")
print(f"  (batch_size, seq_len, d_model)")

In [ ]:
# Appliquer le FFN avec GELU
output_gelu = feed_forward_from_scratch(x_batch, W1, b1, W2, b2, activation="gelu")

print(f"\nRésultat avec GELU:")
print(f"  Input shape:  {x_batch.shape}")
print(f"  Output shape: {output_gelu.shape}")
print(f"  ✓ Dimensions préservées: {x_batch.shape == output_gelu.shape}")

# Appliquer le FFN avec ReLU pour comparaison
output_relu = feed_forward_from_scratch(x_batch, W1, b1, W2, b2, activation="relu")

print(f"\nRésultat avec ReLU:")
print(f"  Input shape:  {x_batch.shape}")
print(f"  Output shape: {output_relu.shape}")
print(f"  ✓ Dimensions préservées: {x_batch.shape == output_relu.shape}")

# Comparer les sorties
diff = np.abs(output_gelu - output_relu).mean()
print(f"\nDifférence moyenne entre GELU et ReLU: {diff:.6f}")
print(f"  (Les activations différentes produisent des sorties différentes)")

## 3. Implémentation PyTorch (nn.Module)

Maintenant, implémentons le FFN avec PyTorch de manière professionnelle.

### 3.1 Création du Module FFN

In [ ]:
# Paramètres
d_model = 256
d_ff = 1024  # 4 × d_model
dropout = 0.1

# Créer le module FFN
ffn = FeedForward(d_model=d_model, d_ff=d_ff, dropout=dropout, activation="gelu")

print("=" * 70)
print("IMPLÉMENTATION PYTORCH: Feed-Forward Network")
print("=" * 70)
print(f"\nConfiguration:")
print(f"  - d_model: {d_model}")
print(f"  - d_ff: {d_ff}")
print(f"  - dropout: {dropout}")
print(f"  - activation: GELU")

print(f"\nArchitecture du module:")
print(ffn)

# Compter les paramètres
total_params = sum(p.numel() for p in ffn.parameters())
print(f"\nNombre total de paramètres: {total_params:,}")
print(f"  - linear1: {d_model * d_ff + d_ff:,} (W1 + b1)")
print(f"  - linear2: {d_ff * d_model + d_model:,} (W2 + b2)")

### 3.2 Explication des Méthodes PyTorch

#### nn.Linear(in_features, out_features)
- **Fonction:** Transformation linéaire $y = xW^T + b$
- **Initialisation:** Xavier uniform par défaut
- **GPU:** Optimisé pour calcul parallèle
- **Gradient:** Calcul automatique avec autograd

#### nn.GELU()
- **Fonction:** $\text{GELU}(x) = x \cdot \Phi(x)$
- **Avantage:** Plus lisse que ReLU
- **Usage:** BERT, GPT-2, GPT-3
- **Propriété:** Gradient non-nul pour valeurs négatives

#### nn.Dropout(p)
- **Fonction:** Désactive aléatoirement des neurones
- **Entraînement:** Actif (dropout appliqué)
- **Évaluation:** Inactif (pas de dropout)
- **Effet:** Réduit le surapprentissage

#### nn.Sequential()
- **Fonction:** Empile des couches séquentiellement
- **Avantage:** Code plus concis
- **Usage:** Pour architectures linéaires simples

In [ ]:
# Créer un input de test
batch_size = 4
seq_len = 10
x = torch.randn(batch_size, seq_len, d_model)

print(f"\nTest du forward pass:")
print(f"  Input shape: {x.shape}")

# Mode entraînement (avec dropout)
ffn.train()
output_train = ffn(x)
print(f"  Output shape (train): {output_train.shape}")

# Mode évaluation (sans dropout)
ffn.eval()
output_eval = ffn(x)
print(f"  Output shape (eval): {output_eval.shape}")

print(f"\n  ✓ Dimensions préservées: {x.shape == output_eval.shape}")

# Vérifier que dropout fait une différence
ffn.train()
output_train2 = ffn(x)
diff_train = torch.abs(output_train - output_train2).mean().item()
print(f"\n  Différence entre deux forward passes (train mode): {diff_train:.6f}")
print(f"    (Non-zéro car dropout est aléatoire)")

ffn.eval()
output_eval2 = ffn(x)
diff_eval = torch.abs(output_eval - output_eval2).mean().item()
print(f"  Différence entre deux forward passes (eval mode): {diff_eval:.10f}")
print(f"    (Zéro car pas de dropout en eval)")

### 3.3 Implémentation Alternative avec nn.Sequential

In [ ]:
# Créer le module FFN avec nn.Sequential
ffn_seq = FeedForwardSequential(d_model=d_model, d_ff=d_ff, dropout=0.0, activation="gelu")

print("=" * 70)
print("IMPLÉMENTATION ALTERNATIVE: nn.Sequential")
print("=" * 70)
print(f"\nArchitecture:")
print(ffn_seq)

# Comparer avec l'implémentation standard
ffn_standard = FeedForward(d_model=d_model, d_ff=d_ff, dropout=0.0, activation="gelu")

# Copier les poids pour avoir les mêmes résultats
ffn_seq.net[0].weight.data = ffn_standard.linear1.weight.data.clone()
ffn_seq.net[0].bias.data = ffn_standard.linear1.bias.data.clone()
ffn_seq.net[3].weight.data = ffn_standard.linear2.weight.data.clone()
ffn_seq.net[3].bias.data = ffn_standard.linear2.bias.data.clone()

# Tester
ffn_standard.eval()
ffn_seq.eval()

x_test = torch.randn(2, 5, d_model)
out_standard = ffn_standard(x_test)
out_seq = ffn_seq(x_test)

diff = torch.abs(out_standard - out_seq).max().item()
print(f"\nComparaison des deux implémentations:")
print(f"  Différence maximale: {diff:.10f}")
print(f"  ✓ Les deux implémentations sont identiques!")

## 4. Visualisation: GELU vs ReLU

Comparons visuellement les deux fonctions d'activation pour comprendre leurs différences.

### 4.1 Fonctions d'Activation

In [ ]:
# Générer les données pour la visualisation
comparison_data = compare_activations()

print("=" * 70)
print("VISUALISATION: GELU vs ReLU")
print("=" * 70)
print("\nGraphiques générés:")
print("  1. Fonctions d'activation (gauche)")
print("  2. Gradients (droite)")
print("\nObservations clés:")
print("  - GELU est lisse partout (pas de discontinuité)")
print("  - ReLU a une discontinuité en x=0")
print("  - GELU permet un petit gradient pour x<0")
print("  - ReLU a gradient=0 pour x<0 (peut 'tuer' des neurones)")

### 4.2 Analyse Détaillée des Différences

#### Comportement pour x < 0

**ReLU:**
- $\text{ReLU}(x) = 0$ pour $x < 0$
- Gradient = 0 pour $x < 0$
- Problème: "Mort" des neurones (gradient nul)

**GELU:**
- $\text{GELU}(x) \approx 0$ pour $x < 0$ mais pas exactement 0
- Gradient $\neq 0$ pour $x < 0$
- Avantage: Tous les neurones peuvent apprendre

#### Comportement pour x > 0

**ReLU:**
- $\text{ReLU}(x) = x$ pour $x > 0$
- Gradient = 1 pour $x > 0$
- Simple et efficace

**GELU:**
- $\text{GELU}(x) \approx x$ pour $x > 0$
- Gradient $\approx 1$ pour $x > 0$
- Légèrement plus complexe mais plus performant

In [ ]:
# Analyser les valeurs spécifiques
test_values = np.array([-2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0])

gelu_vals = gelu_from_scratch(test_values)
relu_vals = relu_from_scratch(test_values)

print("\nComparaison des valeurs:")
print("=" * 70)
print(f"{'x':>8} | {'GELU(x)':>12} | {'ReLU(x)':>12} | {'Différence':>12}")
print("=" * 70)
for x, g, r in zip(test_values, gelu_vals, relu_vals):
    diff = abs(g - r)
    print(f"{x:>8.2f} | {g:>12.6f} | {r:>12.6f} | {diff:>12.6f}")
print("=" * 70)

print("\nObservations:")
print("  - Pour x < 0: GELU ≠ 0, ReLU = 0")
print("  - Pour x > 0: GELU ≈ ReLU (très proche)")
print("  - Pour x = 0: GELU = 0, ReLU = 0")

### 4.3 Impact sur l'Entraînement

Testons l'impact des deux activations sur un forward pass réel.

In [ ]:
# Créer deux FFN identiques sauf pour l'activation
d_model = 128
d_ff = 512

ffn_gelu = FeedForward(d_model, d_ff, dropout=0.0, activation="gelu")
ffn_relu = FeedForward(d_model, d_ff, dropout=0.0, activation="relu")

# Copier les poids pour une comparaison équitable
ffn_relu.linear1.weight.data = ffn_gelu.linear1.weight.data.clone()
ffn_relu.linear1.bias.data = ffn_gelu.linear1.bias.data.clone()
ffn_relu.linear2.weight.data = ffn_gelu.linear2.weight.data.clone()
ffn_relu.linear2.bias.data = ffn_gelu.linear2.bias.data.clone()

ffn_gelu.eval()
ffn_relu.eval()

# Créer un input avec des valeurs positives et négatives
x = torch.randn(1, 10, d_model)

# Forward pass
with torch.no_grad():
    out_gelu = ffn_gelu(x)
    out_relu = ffn_relu(x)

print("=" * 70)
print("IMPACT SUR LE FORWARD PASS")
print("=" * 70)
print(f"\nInput:")
print(f"  Shape: {x.shape}")
print(f"  Mean: {x.mean().item():.6f}")
print(f"  Std: {x.std().item():.6f}")
print(f"  % valeurs négatives: {(x < 0).float().mean().item() * 100:.1f}%")

print(f"\nOutput avec GELU:")
print(f"  Mean: {out_gelu.mean().item():.6f}")
print(f"  Std: {out_gelu.std().item():.6f}")
print(f"  % valeurs négatives: {(out_gelu < 0).float().mean().item() * 100:.1f}%")

print(f"\nOutput avec ReLU:")
print(f"  Mean: {out_relu.mean().item():.6f}")
print(f"  Std: {out_relu.std().item():.6f}")
print(f"  % valeurs négatives: {(out_relu < 0).float().mean().item() * 100:.1f}%")

diff = torch.abs(out_gelu - out_relu).mean().item()
print(f"\nDifférence moyenne: {diff:.6f}")
print(f"  (Les activations différentes produisent des sorties différentes)")

## 5. Validation des Dimensions

Vérifions que le FFN préserve bien les dimensions d'entrée.

### 5.1 Shape Checks Systématiques

In [ ]:
# Test avec différentes configurations
configs = [
    {"d_model": 128, "d_ff": 512, "batch_size": 2, "seq_len": 10},
    {"d_model": 256, "d_ff": 1024, "batch_size": 4, "seq_len": 20},
    {"d_model": 512, "d_ff": 2048, "batch_size": 8, "seq_len": 50},
]

print("=" * 70)
print("VALIDATION DES DIMENSIONS")
print("=" * 70)

for i, config in enumerate(configs, 1):
    print(f"\n[Test {i}] Configuration: d_model={config['d_model']}, d_ff={config['d_ff']}")
    
    # Créer le module
    ffn = FeedForward(config['d_model'], config['d_ff'], dropout=0.0)
    ffn.eval()
    
    # Créer l'input
    x = torch.randn(config['batch_size'], config['seq_len'], config['d_model'])
    
    # Forward pass
    output = ffn(x)
    
    # Vérifications
    assert output.shape == x.shape, f"Shape mismatch: {output.shape} != {x.shape}"
    assert output.size(0) == config['batch_size'], "Batch size mismatch"
    assert output.size(1) == config['seq_len'], "Sequence length mismatch"
    assert output.size(2) == config['d_model'], "Model dimension mismatch"
    
    print(f"  Input:  {x.shape}")
    print(f"  Output: {output.shape}")
    print(f"  ✓ Toutes les vérifications passées!")

print(f"\n{'='*70}")
print("✓ Tous les tests de dimension sont passés avec succès!")
print(f"{'='*70}")

### 5.2 Validation Complète avec la Fonction Utilitaire

In [ ]:
# Utiliser la fonction de validation complète
validate_dimension_preservation(
    d_model=256,
    d_ff=1024,
    batch_size=4,
    seq_len=16
)

## 6. Exercices Pratiques

### Exercice 1: Expérimenter avec d_ff

Testez différents ratios d_ff/d_model et observez l'impact sur le nombre de paramètres.

In [ ]:
# TODO: Complétez ce code
d_model = 256

# Testez différents ratios
ratios = [1, 2, 4, 8]

print("=" * 70)
print("EXERCICE 1: Impact du ratio d_ff/d_model")
print("=" * 70)
print(f"\n{'Ratio':>8} | {'d_ff':>8} | {'Paramètres':>15} | {'Ratio params':>15}")
print("=" * 70)

base_params = None
for ratio in ratios:
    d_ff = d_model * ratio
    
    # TODO: Créez un FFN et comptez les paramètres
    ffn = FeedForward(d_model, d_ff, dropout=0.0)
    total_params = sum(p.numel() for p in ffn.parameters())
    
    if base_params is None:
        base_params = total_params
    
    ratio_params = total_params / base_params
    print(f"{ratio:>8} | {d_ff:>8} | {total_params:>15,} | {ratio_params:>15.2f}x")

print("=" * 70)
print("\nObservation: Le nombre de paramètres augmente quadratiquement avec le ratio!")

### Exercice 2: Comparer les Temps de Calcul

In [ ]:
import time

# TODO: Comparez les temps de calcul pour GELU vs ReLU
d_model = 512
d_ff = 2048
batch_size = 32
seq_len = 128

x = torch.randn(batch_size, seq_len, d_model)

# FFN avec GELU
ffn_gelu = FeedForward(d_model, d_ff, dropout=0.0, activation="gelu")
ffn_gelu.eval()

# FFN avec ReLU
ffn_relu = FeedForward(d_model, d_ff, dropout=0.0, activation="relu")
ffn_relu.eval()

# Mesurer le temps pour GELU
n_iterations = 100
start = time.time()
with torch.no_grad():
    for _ in range(n_iterations):
        _ = ffn_gelu(x)
time_gelu = (time.time() - start) / n_iterations

# Mesurer le temps pour ReLU
start = time.time()
with torch.no_grad():
    for _ in range(n_iterations):
        _ = ffn_relu(x)
time_relu = (time.time() - start) / n_iterations

print("=" * 70)
print("EXERCICE 2: Comparaison des temps de calcul")
print("=" * 70)
print(f"\nConfiguration:")
print(f"  - d_model: {d_model}")
print(f"  - d_ff: {d_ff}")
print(f"  - batch_size: {batch_size}")
print(f"  - seq_len: {seq_len}")
print(f"\nRésultats (moyenne sur {n_iterations} itérations):")
print(f"  - GELU: {time_gelu*1000:.3f} ms")
print(f"  - ReLU: {time_relu*1000:.3f} ms")
print(f"  - Ratio: {time_gelu/time_relu:.2f}x")
print(f"\nConclusion: {'GELU est plus lent' if time_gelu > time_relu else 'ReLU est plus lent'}")

### Exercice 3: Visualiser l'Impact du Dropout

In [ ]:
# TODO: Visualisez l'effet du dropout sur les activations
d_model = 128
d_ff = 512
dropout_rates = [0.0, 0.1, 0.3, 0.5]

x = torch.randn(1, 10, d_model)

print("=" * 70)
print("EXERCICE 3: Impact du dropout")
print("=" * 70)

for dropout in dropout_rates:
    ffn = FeedForward(d_model, d_ff, dropout=dropout)
    ffn.train()  # Mode entraînement pour activer le dropout
    
    # Faire plusieurs forward passes
    outputs = []
    for _ in range(10):
        with torch.no_grad():
            out = ffn(x)
            outputs.append(out)
    
    # Calculer la variance entre les passes
    outputs_stacked = torch.stack(outputs)
    variance = outputs_stacked.var(dim=0).mean().item()
    
    print(f"\nDropout = {dropout:.1f}:")
    print(f"  Variance entre passes: {variance:.6f}")
    print(f"  (Plus de dropout = plus de variance)")

print(f"\n{'='*70}")
print("Observation: Le dropout augmente la variance entre les forward passes!")
print(f"{'='*70}")

## 7. Résumé et Points Clés

### Ce que nous avons appris

1. **Architecture du FFN:**
   - Deux transformations linéaires avec activation non-linéaire
   - Expansion puis projection (d_model → d_ff → d_model)
   - Préservation des dimensions d'entrée

2. **Implémentation from scratch:**
   - Multiplication matricielle explicite avec NumPy
   - Calcul manuel de GELU et ReLU
   - Compréhension des transformations de dimensions

3. **Implémentation PyTorch:**
   - Utilisation de nn.Linear, nn.GELU, nn.Dropout
   - Gestion automatique des gradients
   - Mode train vs eval pour le dropout

4. **GELU vs ReLU:**
   - GELU est plus lisse (pas de discontinuité)
   - GELU permet un gradient non-nul pour x < 0
   - GELU est utilisé dans BERT et GPT
   - ReLU est plus simple mais peut "tuer" des neurones

5. **Validation:**
   - Shape checks à chaque étape
   - Vérification de la préservation des dimensions
   - Tests avec différentes configurations

### Rôle dans le Transformer

Le FFN est appliqué après l'attention dans chaque TransformerBlock:

```
x → Multi-Head Attention → Add & Norm → FFN → Add & Norm → output
```

**Pourquoi le FFN est important:**
- Ajoute de la capacité de transformation non-linéaire
- Permet au modèle d'apprendre des représentations complexes
- Appliqué indépendamment à chaque position ("position-wise")
- Représente ~2/3 des paramètres d'un TransformerBlock

### Prochaines Étapes

Dans le prochain notebook, nous allons:
1. Assembler le TransformerBlock complet
2. Combiner Multi-Head Attention + FFN
3. Ajouter LayerNorm et connexions résiduelles
4. Créer un bloc transformer fonctionnel

**Continuez vers:** `02_transformer_block.ipynb`

## 8. Références

### Articles Scientifiques

1. **Attention Is All You Need** (Vaswani et al., 2017)
   - Paper original des Transformers
   - Décrit l'architecture FFN

2. **BERT: Pre-training of Deep Bidirectional Transformers** (Devlin et al., 2018)
   - Utilise GELU dans le FFN
   - Démontre l'efficacité de GELU

3. **Gaussian Error Linear Units (GELUs)** (Hendrycks & Gimpel, 2016)
   - Introduction de l'activation GELU
   - Analyse théorique et empirique

### Ressources Additionnelles

- PyTorch Documentation: https://pytorch.org/docs/stable/nn.html
- The Illustrated Transformer: http://jalammar.github.io/illustrated-transformer/
- Annotated Transformer: http://nlp.seas.harvard.edu/annotated-transformer/